# VQE Tutorial (Part 1)

In this tutorial, we are going to be working with the hydrogen molecule $H_2$. We will only use ideal simulator (mimicking quantum computers without errors) to run our experiments. 

If you have any difficulties/questions/concerns, shoot an email to eag190@scarletmail.rutgers.edu

### Contents
1. Single Distance
2. Multiple Distances

*By distance, I am referring to the interatomic separation of the 2 hydrogen atoms.*


## 1. Single Distance, Ideal Simulation
Here, we will be analyzing what the different parts of the VQE algo look ,

### 1.1 Some Annoying Import Statements first!

Here are some default things we have to import for any Qiskit program

In [ ]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()

Now we import some VQE-specific things

In [ ]:
from qiskit.aqua.algorithms import VQE, ExactEigensolver
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from qiskit.chemistry.components.variational_forms import UCCSD
from qiskit.aqua.components.variational_forms import RYRZ
from qiskit.chemistry.components.initial_states import HartreeFock
from qiskit.aqua.components.optimizers import COBYLA, SPSA, SLSQP
from qiskit import IBMQ, BasicAer, Aer
from qiskit.chemistry.drivers import PySCFDriver, UnitsType
from qiskit.chemistry import FermionicOperator
from qiskit.providers.aer import noise
from qiskit.aqua import QuantumInstance
from qiskit.ignis.mitigation.measurement import CompleteMeasFitter
from qiskit.aqua.operators import Z2Symmetries
from qiskit.providers.aer.noise import NoiseModel

In [ ]:
import warnings
warnings.filterwarnings("ignore")

### 1.2 Qubit Hamiltonian for $H_2$
Here we will get all the info about the hamiltonian and encode that info into quantum gates


In [ ]:
def get_qubit_ops(dist): 
    # We will get all info we need about the molecule from the PySCF library. Simply, there are 
    # ... some really hardworking chemists that have done most of the work for us!
    mol = 'H .0 .0 .0 ; H .0 .0 {}'
    driver = PySCFDriver(mol.format(dist), unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    
    # Choosing our quantum encoding method
    map_type = 'jordan_wigner'
    
    # Here we will separate the terms of the hamiltonian into 2 halves, one that includes 
    # ... single electron (like Kinetic energy of electrons in the atom) and one that requires 
    # ... information about 2 electrons ( like electron-electron repulsion energy)
    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    
    # When doing VQE, we are largely concerned with the behavior of tiny electrons, not the really large 
    # ... nucleons (protons and neutrons). Since nucleons are really really big, we are gonna assume 
    # ... that nucleons don't move and gonna consider the proton-proton repulsion as a constant.
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    repulsion_energy = molecule.nuclear_repulsion_energy
    
    
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    # As you can see, hartree fock comes from PySCF library for free!
    hf_energy = molecule.hf_energy 
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))
    

    # Now we load all the info from PySCF to Qiskit functions
    ferOp = FermionicOperator(h1=h1, h2=h2)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp.chop(10**-10)
    
    # The term below  has to do with adding the constant nuclear repulsion energy to our VQE-computed 
    # ... energy of the electrons.
    shift = repulsion_energy
    print('Number of qubits are : ' + str(qubitOp.num_qubits))
    print(qubitOp.print_details())
    
    return qubitOp, num_particles, num_spin_orbitals, shift, hf_energy

Let's see how this function works!

In [ ]:
#Pick a distance (units in angstroms)
dist = 0.74
#Let's see what the function outputs
qubitOp, num_particles, num_spin_orbitals, shift, hf_energy = get_qubit_ops(dist)

Notice how so many number of terms constitute the Hamiltonian. Each of these Hamiltonians has its own 'importance'. That is represented by the number right next to pauli terms. What this means is that after you calculate lets say the expectation value of $IIII$ as $E_1$ and that of $IIIZ$ as $E_2$, you would combine them as 
$$E = -0.81E_1 + 0.17E_2 + ... $$
where $E$ is the total energy. Now those coefficients are given as complex numbers above but ignore that part.

### 1.3 Initial State
Here we will simply prepare the Hartree Fock State which is just all the electrons in the lowest energy orbitals!

In [ ]:
# Initial STate
initial_state = HartreeFock(
    num_spin_orbitals,
    num_particles,
    'jordan_wigner'
) 

#Let's take a look at how our initial state is represented in the quantum circuit form
circ_hf = initial_state.construct_circuit().decompose().decompose()
circ_hf.draw(output = 'mpl')

You might be wondering what's the U3($\pi$, 0, $\pi$) gate. It's just another way to express the X gate. (IBM's Quantum computers really like to read gate operations in this u1,u2,u3 language instead of X,Y,Z)

What's happening above is that the spin orbitals corresponding to $q_0$ and $q_2$ are occupied i.e. those are the lowest energy spin orbitals as suggested by the Hartree Fock method

### 1.4  Unitary Coupled Cluster Singles and Doubles

Big name but what we are going to do is convert the Hartree Fock(HF) state into a combo of HF state and some excited states.

In [ ]:
# UCCSD Variational Form
var_form = UCCSD(
    num_orbitals=num_spin_orbitals,
    num_particles=num_particles,
    initial_state=initial_state,
    qubit_mapping='jordan_wigner',
    two_qubit_reduction = False
)
# Let's visualize the circuit again
## Pick 3 random parameters
param = [0,0,0]
circ_uccsd = var_form.construct_circuit(param).decompose().decompose()
circ_uccsd.draw(output = 'mpl')

Very Messy Indeed! What is happening here ?!?

There are 4 spin orbitals, indexed 0,1,2,3. Remember spin orbitals corresponding to $q_0, q_2$ are occupied. These 'occupied' spin orbitals are indexed 0 and 2. So spin orbitals 1 and 3 are empty. 

Now in UCCSD, we will only work with single and double excitations. So possible single excitations are 
$0 \longrightarrow 1$ and $2 \longrightarrow 3$. You might be tempted to say why can't we excite electron in orbital 0 to orbital 3. It's because 0 and 1 are spin up orbitals ( as in they only house electrons with spin up) and 2 and 3 are spin down orbitals. Then an electron in spin up orbital can only be excited to a spin up orbital (UCSSD says you can't flip the spins). So 0 to 3 is just not possible.

Now what would be a double excitation? Exciting electrons in 0 and 2 to 1 and 3 repectively at the same time. 2 singles makes up a double!

So there are 3 possible excited configurations. That's why there are 3 parameters, representing the relative 'importance' of each of the configurations.

*Remember in single excitation, only one of the elctrons is excited. The other one stays in its 'lowest energy' spin orbital*



### 1.5 Running VQE

In [ ]:
#choosing our optimizer 
optimizer = COBYLA(maxiter=2000)

#choosing our simulator
simulator = Aer.get_backend('qasm_simulator')

# Running VQE 
## include_custom gets rid of shot noise i.e. if something that should be 50-50 % comes out 49-51 %,
## ... then include_custom corrects it to 50-50 %.
vqe = VQE(qubitOp, var_form, optimizer, include_custom= True)
vqe_energy = vqe.run(simulator)['energy'] + shift

# Finding exact energies to show how off VQE was from exact energy curve
result = ExactEigensolver(qubitOp).run()
exact_energy = result['energy'] + shift



print('Exact Energy is ' + str(exact_energy) + ' Hartree')
print('VQE energy is '+ str(vqe_energy) + ' Hartree')

In [ ]:
# What's the difference between VQE and exact
vqe_energy - exact_energy

Wow! Remember, chemists want accuracy up till $10^{-3}$ Hartree. So VQE did a pretty good job on the simulator here!

# 2. Multiple Distances, Ideal Simulation

In section 1, we only computed VQE at a single distance. Here we will do so at multiple distances. 

What's happening is that you are taking 2 hydrogen atoms and moving them apart. At each step, let's say 0.1 Angstroms, you will use VQE to compute lowest energy at that distance. 

This way you can see what distance gives you the lowest possible energy i.e. how far hydrogen atoms are in the $H_2$ molecule when the molecule is 'stablest'!

### 2.1 More of the Same 
Qubit Hamiltonian without some print statements

In [ ]:
# added 'a' to the func name to differentiate from the previous get_qubit_ops function

def get_qubit_ops_a(dist): 
    # Defining Molecule
    mol = 'H .0 .0 .0 ; H .0 .0 {}'
    driver = PySCFDriver(mol.format(dist), unit=UnitsType.ANGSTROM,
                         charge=0, spin=0, basis='sto3g')
    molecule = driver.run()
    
    # Mapping to Qubit Hamiltonian
    map_type = 'jordan_wigner'

    h1 = molecule.one_body_integrals
    h2 = molecule.two_body_integrals
    
    nuclear_repulsion_energy = molecule.nuclear_repulsion_energy
    repulsion_energy = molecule.nuclear_repulsion_energy
    
    num_particles = molecule.num_alpha + molecule.num_beta
    num_spin_orbitals = molecule.num_orbitals * 2
    hf_energy = molecule.hf_energy 
    
    print("HF energy: {}".format(molecule.hf_energy - molecule.nuclear_repulsion_energy))
    print("# of electrons: {}".format(num_particles))
    print("# of spin orbitals: {}".format(num_spin_orbitals))

    ferOp = FermionicOperator(h1=h1, h2=h2)
    qubitOp = ferOp.mapping(map_type=map_type, threshold=0.00000001)
    qubitOp.chop(10**-10)
    shift = repulsion_energy
    print('Number of qubits are : ' + str(qubitOp.num_qubits))
    
    return qubitOp, num_particles, num_spin_orbitals, shift, hf_energy

### 2.2 Stretching the molecule apart

In [ ]:
# creating a list of distances to run VQE on 
distances = np.arange(0.1, 2.0, 0.1)

#Empty  boxes for storage
exact_energies = []
vqe_energies = []
hf_energies = []

#Optimizer
optimizer = COBYLA(maxiter=100)

#Begin VQE
for dist in distances:
    print('------------Interatomic Distance = '+ str(np.round(dist, 2)) + '--------------')
    qubitOp, num_particles, num_spin_orbitals, shift, hf_energy = get_qubit_ops_a(dist)
    # Finding exact energies to show how off VQE was from exact energy curve
    result = ExactEigensolver(qubitOp).run()
    exact_energies.append(result['energy'] + shift)
    
    # Initial STate 
    initial_state = HartreeFock(
        num_spin_orbitals,
        num_particles,
        'jordan_wigner'
    ) 
    #print(qubitOp.num_qubits)
    # UCCSD Variational Form
    var_form = UCCSD(
        num_orbitals=num_spin_orbitals,
        num_particles=num_particles,
        initial_state=initial_state,
        qubit_mapping='jordan_wigner', 
        two_qubit_reduction = False
    )
    # Running VQE on ideal sim
    vqe = VQE(qubitOp, var_form, optimizer, include_custom = True)
    results = vqe.run(simulator)['energy'] + shift
    vqe_energies.append(results)
    hf_energies.append(hf_energy)
    print("VQE Result:", results, "Exact Energy:", 
          exact_energies[-1])
print("All energies have been calculated")

### 2.3 Looking at the Results

In [ ]:
# plotting the data
plt.plot(distances, exact_energies, label="Exact Energy")
plt.plot(distances, vqe_energies, label="VQE Energy")
plt.plot(distances, hf_energies, label="HF Energy")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.title('Dissassociation Curves for Hydrogen Molecule')
plt.legend()
plt.show()

It seems like HF energy curve (green) is hugging the VQE energy curve (orange). Where is the exact energy curve? Let's investigate!

In [ ]:
energy_difference= []
hf_difference=[]
for i in range(len(vqe_energies)):
    energy_difference = energy_difference + [vqe_energies[i] - exact_energies[i]]
    hf_difference = hf_difference + [hf_energies[i] - exact_energies[i]]

plt.plot(distances, energy_difference, color= 'tab:orange', label="VQE")
plt.plot(distances, hf_difference, color='tab:green', label=" HF")
plt.title('Difference from Exact Energy')
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree)')
plt.legend()
plt.show()

AH! Look at this! As you increase the separation between H atoms, Hartree focks gives you a worse and worse answer. VQE now seems to be 'hugging' the exact energy since the difference seems to be 0 at all distances. Is that the case? Let's zoom in!

In [ ]:
energy_difference= []
for i in range(len(vqe_energies)):
    energy_difference = energy_difference + [vqe_energies[i] - exact_energies[i]]
plt.plot(distances, energy_difference,color='tab:orange', label="Energy difference between VQE and exact")
plt.xlabel('Atomic distance (Angstrom)')
plt.ylabel('Energy (Hartree) ')
plt.legend()
plt.show()

OOOf! So when you zoom down to $10^{-8}$ Hartrees, you start to see differences between VQE and exact. Again, VQE does really awesome. 

But but this is if we had ideal quantum computers. To see what would happen in reality, switch over to the Part 2 of the Tutorial.

Thank You!